In [1]:
from numpy.random import seed
seed(1)

In [2]:
import numpy as np
import pandas as pd

In [3]:
import tensorflow

In [8]:
numbers = pd.read_csv('everyone_model_numbers.csv')
numbers.head()

,Address Matches,Business Phone,CNAM Sitation,City Matches,First Name Match,Landline,Last Name Match,No Address,No Name,Relevant Names,State Match,Workplace Match,Zipcode Matches,label
0,0,1,0,1,1,1,1,0,0,1,1,0,1,connected
1,1,1,0,1,0,1,0,0,0,1,1,0,1,connected
2,1,1,0,1,0,1,0,0,0,1,1,0,1,connected
3,0,1,0,1,0,1,0,0,0,1,1,0,1,connected
4,1,1,0,1,0,1,0,0,0,1,1,0,1,connected


In [9]:
X = numbers.drop("label",axis = 1)
y = numbers['label']
print(X.shape, y.shape)

(101, 13) (101,)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 12, stratify = y)

In [12]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\vigrose\AppData\Local\Programs\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [14]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [15]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=13))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [18]:
model.compile(optimizer ='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1400      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 11,702
Trainable params: 11,702
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(X_train_scaled,
         y_train_categorical,
         epochs=100,
         shuffle=True,
         verbose=2
         )

Epoch 1/100
75/75 - 0s - loss: 0.3235 - acc: 0.8400
Epoch 2/100
75/75 - 0s - loss: 0.3260 - acc: 0.8400
Epoch 3/100
75/75 - 0s - loss: 0.3189 - acc: 0.8533
Epoch 4/100
75/75 - 0s - loss: 0.3216 - acc: 0.8400
Epoch 5/100
75/75 - 0s - loss: 0.3191 - acc: 0.8533
Epoch 6/100
75/75 - 0s - loss: 0.3160 - acc: 0.8533
Epoch 7/100
75/75 - 0s - loss: 0.3123 - acc: 0.8533
Epoch 8/100
75/75 - 0s - loss: 0.3095 - acc: 0.8533
Epoch 9/100
75/75 - 0s - loss: 0.3071 - acc: 0.8533
Epoch 10/100
75/75 - 0s - loss: 0.3060 - acc: 0.8533
Epoch 11/100
75/75 - 0s - loss: 0.3029 - acc: 0.8533
Epoch 12/100
75/75 - 0s - loss: 0.3034 - acc: 0.8533
Epoch 13/100
75/75 - 0s - loss: 0.3016 - acc: 0.8533
Epoch 14/100
75/75 - 0s - loss: 0.2982 - acc: 0.8533
Epoch 15/100
75/75 - 0s - loss: 0.2972 - acc: 0.8667
Epoch 16/100
75/75 - 0s - loss: 0.2972 - acc: 0.8667
Epoch 17/100
75/75 - 0s - loss: 0.2952 - acc: 0.8667
Epoch 18/100
75/75 - 0s - loss: 0.2937 - acc: 0.8667
Epoch 19/100
75/75 - 0s - loss: 0.2936 - acc: 0.8667
Ep

In [26]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f'Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}')

26/26 - 0s - loss: 0.6014 - acc: 0.7692
Normal Neural Network - Loss: 0.6014118790626526, Accuracy: 0.7692307829856873


In [27]:
predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(predictions)

In [28]:
data = {'Predicted': prediction_labels, 'Actual':list(y_test)}
pd.DataFrame(data)

,Predicted,Actual
0,connected,connected
1,connected,disconnected
2,connected,connected
3,disconnected,disconnected
4,disconnected,disconnected
5,disconnected,disconnected
6,disconnected,disconnected
7,connected,connected
8,connected,connected
9,connected,connected


In [39]:
X_test.to_csv("test_ekata.csv")

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(list(y_test), prediction_labels, labels = ['disconnected','connected'])

array([[11,  2],
       [ 4,  9]], dtype=int64)

In [45]:
office_numbers = pd.read_csv('Ekata/ekata_numbers.csv')

In [46]:
x = office_numbers[['OFFICE_TELEPHONE', 'label', 'Index']]

In [50]:
test = pd.read_csv('test_ekata.csv')

In [51]:
test_numbers = pd.merge(test, x, on='Index')
test_numbers['Model_Predictions']=prediction_labels

In [52]:
test_numbers

,Index,Address Match,Ambulatory Industry,Business Phone,City Match,First Name Match,Fixed VOIP,Hospital Industry,Landline,Last Name Match,...,Non Fixed Voip,Nursing Industry,Personal Phone,Relevant Name,State Match,Workplace Match,ZipCode Match,OFFICE_TELEPHONE,label,Model_Predictions
0,11,1,1,1,1,1,0,0,0,1,...,1,0,0,1,1,0,1,5757697365,connected,connected
1,52,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,7273020000,disconnected,disconnected
2,46,0,1,1,1,0,0,0,0,0,...,1,0,0,0,1,0,0,7137721200,connected,connected
3,82,0,1,1,0,0,0,0,1,1,...,0,0,0,1,1,0,1,5152801252,disconnected,disconnected
4,87,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,5202145354,disconnected,disconnected
5,56,0,1,1,0,0,0,0,1,0,...,0,0,0,1,1,0,0,8139727629,disconnected,disconnected
6,54,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,8585739902,disconnected,disconnected
7,5,1,1,1,1,0,0,0,1,0,...,0,0,0,1,1,0,1,4173357222,connected,connected
8,6,1,1,1,0,0,0,0,1,0,...,0,0,0,1,1,0,1,4403124569,connected,disconnected
9,25,0,1,1,0,0,0,0,1,0,...,0,0,0,1,1,0,1,7659833427,connected,disconnected


In [53]:
test_numbers.to_csv("Ekata_Model_Predictions.csv")